In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.types import Row

from pyspark.sql import SQLContext

In [ ]:
spark = SparkContext(master='local', appName="TransformacionesYAcciones")
sqlContext = SQLContext(spark)

In [ ]:
pwd

In [ ]:
!ls ./files/

In [ ]:
path = '/home/contreras/Descargas/Spark/files/'
deportistaOlimpicoRDD = spark.textFile(path+'deportista.csv') \
    .map(lambda line: line.split(","))

deportistaOlimpicoRDD2 = spark.textFile(path+'deportista2.csv') \
    .map(lambda line: line.split(","))

deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [ ]:
deportistaOlimpicoRDD.take(5)

In [ ]:
#Retiramos encabezado

def eliminarEncabezado(indice, iterador):
    return iter(list(iterador)[1:])


In [ ]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.\
    mapPartitionsWithIndex(eliminarEncabezado)

In [ ]:
deportistaOlimpicoRDD.take(5)

In [ ]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: (
    int(1[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    float(l[5]),
    int(l[6]),
    ))

In [ ]:
deportistaOlimpicoRDD.take(5)

In [ ]:
Schema = StructType([
    StructField('deportista_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', IntegerType(), False),
    StructField('edad', IntegerType(), False),
    StructField('altura', IntegerType(), False),
    StructField('peso', FloatType(), False),
    StructField('edad', IntegerType(), False)
])

In [ ]:
deportistaDF = sqlContext.createDataFrame(deportistaOlimpico, schema)

In [ ]:
deportistaDF.show(5)

In [ ]:
#RETO CREAR LOS DATA FRAME RESTANTES

In [ ]:
# Paises Equipos

equiposilimpicosRDD = spark.textFile(path+'paises.csv')\
    .map(lambda line: line.split(","))\
    .mapPartitionsWithindex(eliminarEncabezado) \
    .map(lambda l : (
    
    int(l[0]),
    l[1],
    l[2],
))

equiposSchema = StructType([
    StructField('id', IntegerType(), False),
    StructField('equipo', StringType(), False),
    StructFiel('sigla', StringType(), False),
])

paisesDf = sqlContext.createDataFrame(equiposolimpicosRDD, equiposSchema)

paisesDF.show(5)


In [ ]:
# Deportes

Schema = Structype([
    StructFiel('deporte_id', Integertype(), False),
    StructField('deporte', StringType(), False)
])


# Paso 2 Crea DATAFRAME con la lectura del Schema

deportesDF = sqlContext.read.schema(deporteSchema) \
    .option('header', 'true').csv(path+'deporte.csv')

deportesDF.show(5)

In [ ]:
# Evento

Schema = Structype([
    StructFiel('evento_id', Integertype(), False),
    StructField('nombre', StringType(), False),
    StructFiel('deporte_id', Integertype(), False),

])


# Paso 2 Crea DATAFRAME con la lectura del Schema

deportesOlimpicosDF = sqlContext.read.schema(eventoSchema) \
    .option('header', 'true').csv(path+'evento.csv')

ddeportesOlimpicosDF.show(5)

In [ ]:
#OPERACIONES SOBRE DF

In [ ]:
# ver el Schema
deportesDF.printSchema()

In [ ]:
deportistaDF.printSchema()

In [ ]:
# RENOMBRADO DE COLUMNAS

deportistaOlimpicoDF = deportistaDF \

    .withColumnRenamed("genero", 'sexo') \
    .drop('altura')

In [ ]:
deportistaolimpicoDF.printSchema()

In [ ]:
#Usando el Rey SELECT

from pyspark.sql.functions import *

deportistaOlimpicoDF = deportistaOlimpicoDF.select('deportista_id','r',
                                                  col("edad").alias("edadAljugador"),
                                                   "equipo_id"
                                                  )

In [ ]:
deportistaOlimpicoDF.show(5)

In [ ]:
# USANDO filter

deportistaOlimpicoDF.sort("edadAljugador").show()

In [ ]:
# Agrupaciones y operaciones join sobre DF

In [ ]:
deportistaOlimpicoDF.printSchema()

In [ ]:
resultadoDF.printSchema()

In [ ]:
juegoDF.printSchema()

In [ ]:
deportesOlimpicosDF.printSchema()

In [ ]:
paisesDF.printSchema()

In [ ]:
# JOINS

deportistaOlimpicoDF \
    .join(

    resultadDF,
    deportistaOlimpicoDF.deportista_id == resultadoDF.deportista_id,
    "left"
    )\
    .join(
    juegoDF,
    juegoDF.juego_id == resultadoDF.juego_id,
    "left"
    )\
    .join(
    deportistaOlimpicoDF,
    deportistaOlimpicoDF.evento_id == resultadoDF.evento_id,
    "left"
    )\
    .select(deportistaOlimpicoDF.nombre,           
           "edadAlJugar",
            "medalla",
            col("anio").alias("Anio de juego"),
            deportistaOlimpicoDF.nombre.alias("Nombre de disciplina")
           ).show()

In [ ]:
# RETO

In [ ]:
resultadoDF \
    .join(
    deportistaOlimpicoDF,
    resultDF.deportista_id == deportistaOlimpicoDF.deportista_id,
    "left"
    )\
    
    .join(
    paisesDF,
    deportistaOlimpicoDF.equipo_id == paisesDF.id,
    "left"
    )\
    .select("medalla",
           paisesDF.equipo,
            paisesDF.sigla
    )\

    .where(resultadoDF.medalla != "NA").show()

In [ ]:
# Solucion reto joins

In [ ]:
resultadoDF.filter(resultadoDF.medalla != "NA") \
    .join(
    deportistaOlimpicoDF,
    resultDF.deportista_id == deportistaOlimpicoDF.deportista_id,
    "left"
    )\
    
    .join(
    paisesDF,
    deportistaOlimpicoDF.equipo_id == paisesDF.id,
    "left"
    )\
    .select("medalla",
           paisesDF.equipo,
            paisesDF.sigla
    )\
    .sport(col("sigla").desc())\
    .show()

In [ ]:
# Funciones de agrupación

In [ ]:
medallistaXAnio = deportistaOlimpicoDF \
    .join(
        resultadoDF,
        deportistaOlimpicoDF.deportistaDF_id == resultadoDF.deportista_id,
        "left"
    )\
    .join(
    juegoDF,
    juegoDF.juego_id == resultadoDF.juego_id,
    "left"
    )\
    .join(
    paisesDF,
    deportistaOlimpicoDF.equipo_id == paisesDF.id,
    "left"
    )\
    .join(
    deportistaOlimpicoDF,
    resultDF.deportista_id == deportistaOlimpicoDF.deportista_id,
    "left"
    )\
    .join(
    deportesDF,
    deportistaOlimpicoDF,
    resultDF.deportista_id == deportistaOlimpicoDF.deportista_id,
    "left"
    )\
    .select(
    "sigla",
    "anio",
    "medalla",
    deportesolimpicosDF.nombre.alias("Nombre subdisciplina"),
    deportesDF.deporte.alias("Nombre disciplina")
    deportistaOlimpicoDf.nombre
    )


In [ ]:
medallistaSAnio.show()

In [ ]:
# DESCARTAMOS DEPORTISTAS SIN MEDALLA

medallistaXAnio2 = medallistaXAnio.filter(resultadoDF.medalla != "NA")\
    .sort("anio")\
    .groupBy("sigla", "anio", "Nombre subdisciplina") \
    .count()

medallistaXAnio2.show()


In [ ]:
medallistaXANio2.printSchema()

In [ ]:
medallistaXAnio2.groupBy("Sigla", "anio") \
    .agg(sum("count").alias("Total de medallas"), \
        avg("count").alias("MEdallas promedio")).show()

# .agg() o agrevobe es la forma recomendada por la documentacion

In [ ]:
# SQL

In [ ]:
# REGISTRO DE DATAFRAMES COMO TABLAS SQL

resultadoDF.registerTempTable("resultado")
deportistaOlimpicoDF.registerTempTable("deportista")
paisesDF.registerTemTable("paises")

In [ ]:
sqlContext.sql("SELECT * FROM deportista").show(5)

In [ ]:
sqlContext.sql("""
    SELECT medalla, equipo, sigla FROM resultado as r
    JOIN deportista as d
    ON r.deportista_id = d.deportista_id
    JOIN paises as p
    ON p.id = d.equipo_id
    WHERE medalla <> "NA"
    ORDER BY sigla DESC
    """).show(15)

In [3]:
spark

NameError: name 'spark' is not defined

In [ ]:
# UDF

In [2]:
!ls ./files/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [1]:
!head -n 5 ./files/deportistaError.csv

deportista_id,nombre,genero,edad,altura,peso,equipo_id
1,A Dijiang,1,24,180,80,199
2,A Lamusi,1,23,170,60,199
3,Gunnar Nielsen Aaby,1,24,,,273
4,Edgar Lindenau Aabye,1,34,,,278


In [ ]:
DeportistaError_schema = StructType([
    StructField("deportista_id", StringType(), False),
    StructField("nombre", StringType(), False),
    StructField("genero", StringType(), False),
    StructField("edad", StringType(), False),
    StructField("altura", StringType(), False),
    StructField("peso", StringType(), False),
    StructField("equipo_id", StringType(), False)
])

deportistaErrorDF = sqlContext.read.format("csv").\
    option("header", True).\
    schema(DeportistaError_schema).\
    load(path+"deportistaError.csv")

In [ ]:
deportistaErrorDF.show()

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
def conversionEnteros(valor):
    try:
        return int(valor) if len(valor) > o else 0
    except tyError as e:
        return 0
    
conversionEnteros_udf = udf(lambda z : conversionEnteros(z), IntegerType())
sqlContext.udf.register("conversionEnteros_udf", conversionEnteros_udf)

DeportistaErrorDF.select(conversionEnteros_udf("altura")
    .alias("alturaUDF")).show()

In [ ]:
# Comprendiendo la persistencia y particionado

In [ ]:
#DF y Replicacion

In [ ]:
from pyspark.storagelevel import StorageLevel

In [ ]:
medallistaXAnio.is_cached

In [ ]:
# Creamos la persistencia
medallistaXAnio.rdd.cache()

In [ ]:
# Saber el tiṕo de persistencia y nivel de replicacion

medallistaXAnio.rdd.getStorageLevel()

In [ ]:
# Eliminando la persistencia
medallistaXAnio.rdd.unpersist()

In [ ]:
# Usando persistencia MEMORY AND DISK con replicacion 2

medallistaXAnio.rdd.persist(Storagelevel.MEMORY_AND_DISK_2)


In [ ]:
# Saber el tipo de persistencia y nivel de recopilacion
medallistaXAnio.rdd.getStorageLevel()

In [ ]:
# Eliminando la persistencia
medallistaXAnio.rdd.unpersist()


In [ ]:
# Definiendo un storageLevel custom
StorageLevel.MEMORY_AND_DISK_# = Storagelevel(True, True, False, 3)

In [ ]:
# Persistimos el DATAFRAME
medallistaXAnio.rdd.persist(StorageLevel.MEMORY_AND_DISK_3)

In [ ]:
# Saber el tipo de persistencia y nivel de recopilacion

medallistaXAnio.rdd..getStorageLevel()

In [ ]:
# Particionando datos

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Particionado") \
    .master("local[5]").getOrCreate()

In [ ]:
df = spark.range(0,20)
df.rdd.getnu,Partitions()

In [ ]:
redd1 = sparkContext.parallelize((0,20), 10)
redd1.getNumPartitions()

In [ ]:
rddDesdeArcivo = spark \
    .sparkContext \
    .textFile("./files/deportes.csv", 10)

In [ ]:
rddDesdeArchivo.getnu,partitions()

In [ ]:
rddDesdeArchivo.saveAsTextFile("./partition_example/salidadetexto")

In [ ]:
!ls "./partition_example/salidadetexto/"

In [ ]:
# Verificamos los archivos particionados

!head -n 5 ./partition_example/salidadetexto/part-00004

In [ ]:
rdd = spark.sparkContext.wholeTextFile('./partition_example/salidadetexto/*')

In [ ]:
rdd.take(2)

In [ ]:
# FORMA SENCILLA CARGAR
lista = rdd.mapValues(lambda x: x.split()).collect()

In [ ]:
lista

In [ ]:
lista = [1[0] for l in lista]
lista.sort()

In [ ]:
rdd2 = spark \
    .sparkContext \
    .textFile(','.join(lista), 10).map(lambda l: l.split(","))

In [ ]:
rdd2.take(5)